In [5]:
pip install pyspark py4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=0f58c11c2ece6198b2a69d4990294cb1da6ad08eaf72dce0b5f044dbcc1991c2
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [6]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()
spark = SparkSession.builder.appName("TempViews").getOrCreate()

In [7]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
home_sales_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
home_sales_df.show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [8]:
# 2. Create a temporary view of the DataFrame.
home_sales_df.createOrReplaceTempView('home_sales')


In [9]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
query = """SELECT date_built,
           ROUND(AVG(price), 2) AS AvgPrice
           FROM home_sales
           GROUP BY date_built
           ORDER BY date_built DESC"""
spark.sql(query).show()


+----------+---------+
|date_built| AvgPrice|
+----------+---------+
|      2017|315799.06|
|      2016|315471.57|
|      2015|315322.64|
|      2014|316395.73|
|      2013| 317597.6|
|      2012|314072.77|
|      2011|314804.32|
|      2010|309751.17|
+----------+---------+



In [10]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
query = """SELECT date_built,
           ROUND(AVG(price), 2) AS AvgPrice
           FROM home_sales
           WHERE bedrooms = '3' AND bathrooms = '3'
           GROUP BY date_built
           ORDER BY date_built DESC"""
spark.sql(query).show()


+----------+---------+
|date_built| AvgPrice|
+----------+---------+
|      2017|292676.79|
|      2016|290555.07|
|      2015| 288770.3|
|      2014|290852.27|
|      2013|295962.27|
|      2012|293683.19|
|      2011|291117.47|
|      2010|292859.62|
+----------+---------+



In [11]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
query = """SELECT date_built,
           ROUND(AVG(price), 2) AS AvgPrice
           FROM home_sales
           WHERE bedrooms = '3' AND bathrooms = '3' AND floors = 2 AND sqft_living >= 2000
           GROUP BY date_built
           ORDER BY date_built DESC"""
spark.sql(query).show()


+----------+---------+
|date_built| AvgPrice|
+----------+---------+
|      2017|280317.58|
|      2016| 293965.1|
|      2015|297609.97|
|      2014|298264.72|
|      2013|303676.79|
|      2012|307539.97|
|      2011|276553.81|
|      2010|285010.22|
+----------+---------+



In [12]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

query = """SELECT date_built,
           ROUND(AVG(price), 2) AS AvgPrice
           FROM home_sales
           WHERE price >= 350000
           GROUP BY date_built
           ORDER BY date_built DESC"""
spark.sql(query).show()

print("--- %s seconds ---" % (time.time() - start_time))

+----------+---------+
|date_built| AvgPrice|
+----------+---------+
|      2017|475884.72|
|      2016|478550.09|
|      2015|471517.37|
|      2014|475674.82|
|      2013|476837.88|
|      2012|475524.98|
|      2011|472861.91|
|      2010|463301.08|
+----------+---------+

--- 1.6924304962158203 seconds ---


In [13]:
# 7. Cache the the temporary table home_sales.
spark.sql('cache table home_sales')

DataFrame[]

In [14]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [15]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

query = """SELECT date_built,
           ROUND(AVG(price), 2) AS AvgPrice
           FROM home_sales
           WHERE price >= 350000
           GROUP BY date_built
           ORDER BY date_built DESC"""
spark.sql(query).show()

print("--- %s seconds ---" % (time.time() - start_time))


+----------+---------+
|date_built| AvgPrice|
+----------+---------+
|      2017|475884.72|
|      2016|478550.09|
|      2015|471517.37|
|      2014|475674.82|
|      2013|476837.88|
|      2012|475524.98|
|      2011|472861.91|
|      2010|463301.08|
+----------+---------+

--- 0.4424624443054199 seconds ---


In [16]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_sales_df.write.partitionBy("date_built").mode("overwrite").parquet("home_sales_partitioned")

In [17]:
# 11. Read the parquet formatted data.
p_df_p = spark.read.parquet('home_sales_partitioned')

In [18]:
# 12. Create a temporary table for the parquet data.
p_df_p.createOrReplaceTempView('p_home_sales_p')

In [19]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

query = """SELECT date_built,
           ROUND(AVG(price), 2) AS AvgPrice
           FROM p_home_sales_p
           WHERE price >= 350000
           GROUP BY date_built
           ORDER BY date_built DESC"""
spark.sql(query).show()

print("--- %s seconds ---" % (time.time() - start_time))

+----------+---------+
|date_built| AvgPrice|
+----------+---------+
|      2017|475884.72|
|      2016|478550.09|
|      2015|471517.37|
|      2014|475674.82|
|      2013|476837.88|
|      2012|475524.98|
|      2011|472861.91|
|      2010|463301.08|
+----------+---------+

--- 0.9450385570526123 seconds ---


In [20]:
# 14. Uncache the home_sales temporary table.
spark.sql('uncache table home_sales')

DataFrame[]

In [21]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')


False